# Modelos de Ecuaciones Estructurales (SEM) - Variables de Congruencia

Este notebook implementa **modelos de path analysis** para analizar la relación entre las variables ideológicas y las variables de **congruencia ideológica** creadas en el Notebook 52.

## Modelo para cada variable dependiente:

```
Indice_Progresismo ────→ Y
                          ↑
Indice_Conservadurismo ──→
```

**Ecuación:**
```
Y = β₀ + β₁(Indice_Progresismo) + β₂(Indice_Conservadurismo) + ε
```

## Variables Dependientes (4 modelos):

**Variables Congruentes (ideológicamente consistentes):**
1. `CO_Congruente` = Cambio_Op_Sum_Pro_Izq + Cambio_Op_Sum_Con_Der
2. `CT_Congruente` = Cambio_Tiempo_Sum_Pro_Izq + Cambio_Tiempo_Sum_Con_Der

**Variables Incongruentes (ideológicamente inconsistentes):**
3. `CO_Incongruente` = Cambio_Op_Sum_Pro_Der + Cambio_Op_Sum_Con_Izq
4. `CT_Incongruente` = Cambio_Tiempo_Sum_Pro_Der + Cambio_Tiempo_Sum_Con_Izq

## Hipótesis a Probar:

**H1:** Mayor progresismo predice **más cambios congruentes**  
**H2:** Mayor conservadurismo predice **más cambios congruentes**  
**H3:** Los índices predicen **mejor** los cambios congruentes que los incongruentes  
**H4:** Las variables congruentes tienen **mayor R²** que las incongruentes

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Imports para SEM
try:
    from semopy import Model
    from semopy.inspector import inspect
    print("✓ semopy disponible")
    USAR_SEMOPY = True
except ImportError:
    print("⚠️  semopy no disponible, usando statsmodels OLS")
    from statsmodels.api import OLS, add_constant
    from sklearn.preprocessing import StandardScaler
    USAR_SEMOPY = False

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

⚠️  semopy no disponible, usando statsmodels OLS


## 1. Cargar Datos

In [ ]:
# Rutas
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

## 2. Definir Variables

In [ ]:
# Variables predictoras
Predictores = [
    'Indice_Progresismo',
    'Indice_Conservadurismo'
]

# Variables dependientes - Congruencia
Outcomes_Congruencia = [
    'CO_Congruente',
    'CO_Incongruente',
    'CT_Congruente',
    'CT_Incongruente'
]

print(f"✓ Variables definidas:")
print(f"  - Predictores: {len(Predictores)}")
print(f"  - Outcomes: {len(Outcomes_Congruencia)}")

## 3. Verificar Existencia de Variables de Congruencia

In [ ]:
print("="*70)
print("VERIFICANDO VARIABLES DE CONGRUENCIA")
print("="*70)

for nombre_df, df in dfs_Finales.items():
    print(f"\n{nombre_df}:")
    
    for var in Outcomes_Congruencia:
        if var in df.columns:
            n_validos = df[var].notna().sum()
            media = df[var].mean()
            print(f"  ✓ {var:<25} n={n_validos:>4}, media={media:>8.2f}")
        else:
            print(f"  ❌ {var:<25} NO ENCONTRADA")

print("\n" + "="*70)

## 4. Función para Ejecutar Modelo SEM

In [ ]:
def Ejecutar_Modelo_Path(df, outcome, predictores, nombre_modelo):
    """
    Ejecuta un modelo de path analysis (regresión múltiple).
    """
    
    # Verificar que las variables existan
    variables_necesarias = [outcome] + predictores
    variables_existentes = [v for v in variables_necesarias if v in df.columns]
    
    if len(variables_existentes) != len(variables_necesarias):
        faltantes = set(variables_necesarias) - set(variables_existentes)
        print(f"  ⚠️  Variables faltantes en {nombre_modelo}: {faltantes}")
        return None
    
    # Seleccionar datos y eliminar NaN
    df_modelo = df[variables_necesarias].dropna()
    
    if len(df_modelo) < 10:
        print(f"  ⚠️  Datos insuficientes en {nombre_modelo}: n={len(df_modelo)}")
        return None
    
    # Usar OLS de statsmodels
    try:
        from statsmodels.api import OLS, add_constant
        from sklearn.preprocessing import StandardScaler
        
        # Preparar datos
        X = df_modelo[predictores]
        y = df_modelo[outcome]
        
        # Agregar constante
        X_const = add_constant(X)
        
        # Ajustar modelo
        modelo_ols = OLS(y, X_const).fit()
        
        # Calcular coeficientes estandarizados
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_std = scaler_X.fit_transform(X)
        y_std = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
        
        modelo_std = OLS(y_std, X_std).fit()
        
        # Formatear resultados
        resultados = {
            'Modelo': nombre_modelo,
            'Outcome': outcome,
            'n': len(df_modelo),
            'R²': modelo_ols.rsquared,
            'R²_ajustado': modelo_ols.rsquared_adj,
            'AIC': modelo_ols.aic,
            'BIC': modelo_ols.bic,
            'F_stat': modelo_ols.fvalue,
            'F_pvalue': modelo_ols.f_pvalue,
            'Coeficientes': {}
        }
        
        # Extraer coeficientes (excluyendo constante)
        for i, pred in enumerate(predictores):
            resultados['Coeficientes'][pred] = {
                'β': modelo_ols.params[pred],
                'β_std': modelo_std.params[i],
                'SE': modelo_ols.bse[pred],
                't': modelo_ols.tvalues[pred],
                'p': modelo_ols.pvalues[pred]
            }
        
        return resultados
        
    except Exception as e:
        print(f"  ❌ Error en OLS para {nombre_modelo}: {e}")
        return None

## 5. Ejecutar Modelos para GENERALES

In [ ]:
print("="*70)
print("EJECUTANDO MODELOS SEM: GENERALES (Variables de Congruencia)")
print("="*70)

resultados_generales = []

for outcome in Outcomes_Congruencia:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Generales,
        outcome,
        Predictores,
        f"Generales_{outcome}"
    )
    
    if resultado:
        resultados_generales.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  R²_ajustado = {resultado['R²_ajustado']:.4f}")
        print(f"  F({len(Predictores)}, {resultado['n']-len(Predictores)-1}) = {resultado['F_stat']:.2f}, p = {resultado['F_pvalue']:.4f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs['β']
            beta_std = coefs['β_std']
            p_val = coefs['p']
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  β_std = {beta_std:>6.3f}  (p = {p_val:.4f}) {sig}")
        
        if resultado['F_pvalue'] < 0.05:
            print(f"\n  ✅ Modelo globalmente significativo (F-test p < 0.05)")
        else:
            print(f"\n  ⚠️  Modelo NO significativo globalmente (F-test p >= 0.05)")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_generales)} modelos ejecutados para Generales")
print("="*70)

## 6. Ejecutar Modelos para BALLOTAGE

In [ ]:
print("="*70)
print("EJECUTANDO MODELOS SEM: BALLOTAGE (Variables de Congruencia)")
print("="*70)

resultados_ballotage = []

for outcome in Outcomes_Congruencia:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Ballotage,
        outcome,
        Predictores,
        f"Ballotage_{outcome}"
    )
    
    if resultado:
        resultados_ballotage.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  R²_ajustado = {resultado['R²_ajustado']:.4f}")
        print(f"  F({len(Predictores)}, {resultado['n']-len(Predictores)-1}) = {resultado['F_stat']:.2f}, p = {resultado['F_pvalue']:.4f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs['β']
            beta_std = coefs['β_std']
            p_val = coefs['p']
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  β_std = {beta_std:>6.3f}  (p = {p_val:.4f}) {sig}")
        
        if resultado['F_pvalue'] < 0.05:
            print(f"\n  ✅ Modelo globalmente significativo (F-test p < 0.05)")
        else:
            print(f"\n  ⚠️  Modelo NO significativo globalmente (F-test p >= 0.05)")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_ballotage)} modelos ejecutados para Ballotage")
print("="*70)

## 7. Crear Tablas de Resultados

In [ ]:
def Crear_Tabla_Resultados(lista_resultados):
    """
    Convierte lista de resultados en DataFrames.
    """
    
    # Tabla de métricas
    metricas_data = []
    for res in lista_resultados:
        metricas_data.append({
            'Outcome': res['Outcome'],
            'n': res['n'],
            'R²': res['R²'],
            'R²_ajustado': res['R²_ajustado'],
            'AIC': res['AIC'],
            'BIC': res['BIC'],
            'F_stat': res['F_stat'],
            'F_pvalue': res['F_pvalue']
        })
    
    df_metricas = pd.DataFrame(metricas_data)
    
    # Tabla de coeficientes
    coef_data = []
    for res in lista_resultados:
        for pred, coefs in res['Coeficientes'].items():
            p_val = coefs['p']
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            coef_data.append({
                'Outcome': res['Outcome'],
                'Predictor': pred,
                'β': coefs['β'],
                'β_std': coefs['β_std'],
                'SE': coefs['SE'],
                't': coefs['t'],
                'p-valor': p_val,
                'Sig': sig
            })
    
    df_coeficientes = pd.DataFrame(coef_data)
    
    return df_metricas, df_coeficientes

In [ ]:
# Crear tablas
df_metricas_gen, df_coef_gen = Crear_Tabla_Resultados(resultados_generales)
df_metricas_bal, df_coef_bal = Crear_Tabla_Resultados(resultados_ballotage)

print("\n📋 MÉTRICAS - GENERALES:")
print(df_metricas_gen.to_string(index=False))

print("\n📋 COEFICIENTES - GENERALES:")
print(df_coef_gen.to_string(index=False))

print("\n📋 MÉTRICAS - BALLOTAGE:")
print(df_metricas_bal.to_string(index=False))

print("\n📋 COEFICIENTES - BALLOTAGE:")
print(df_coef_bal.to_string(index=False))

## 8. Guardar Resultados en Excel

In [ ]:
# Crear carpeta
Carpeta_Resultados = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_SEM')
if not os.path.exists(Carpeta_Resultados):
    os.makedirs(Carpeta_Resultados)

# Guardar Generales
archivo_gen = os.path.join(Carpeta_Resultados, 'SEM_Variables_Congruencia_Generales.xlsx')
with pd.ExcelWriter(archivo_gen, engine='openpyxl') as writer:
    df_metricas_gen.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_gen.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ SEM_Variables_Congruencia_Generales.xlsx")

# Guardar Ballotage
archivo_bal = os.path.join(Carpeta_Resultados, 'SEM_Variables_Congruencia_Ballotage.xlsx')
with pd.ExcelWriter(archivo_bal, engine='openpyxl') as writer:
    df_metricas_bal.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_bal.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ SEM_Variables_Congruencia_Ballotage.xlsx")

## 9. Análisis Comparativo: Congruente vs Incongruente

In [ ]:
print("="*70)
print("COMPARACIÓN: CONGRUENTE vs INCONGRUENTE")
print("="*70)

print("\n📊 GENERALES - Comparación de R²:")
print("-"*70)

for tipo in ['CO', 'CT']:
    congruente = df_metricas_gen[df_metricas_gen['Outcome'] == f'{tipo}_Congruente']
    incongruente = df_metricas_gen[df_metricas_gen['Outcome'] == f'{tipo}_Incongruente']
    
    if len(congruente) > 0 and len(incongruente) > 0:
        r2_cong = congruente['R²'].values[0]
        r2_incong = incongruente['R²'].values[0]
        
        print(f"\n  {tipo}:")
        print(f"    Congruente:    R² = {r2_cong:.4f}")
        print(f"    Incongruente:  R² = {r2_incong:.4f}")
        print(f"    Diferencia:    {r2_cong - r2_incong:.4f}")
        
        if r2_cong > r2_incong:
            print(f"    ✅ Congruente mejor explicado por ideología")
        else:
            print(f"    ⚠️  Incongruente mejor explicado")

print("\n📊 BALLOTAGE - Comparación de R²:")
print("-"*70)

for tipo in ['CO', 'CT']:
    congruente = df_metricas_bal[df_metricas_bal['Outcome'] == f'{tipo}_Congruente']
    incongruente = df_metricas_bal[df_metricas_bal['Outcome'] == f'{tipo}_Incongruente']
    
    if len(congruente) > 0 and len(incongruente) > 0:
        r2_cong = congruente['R²'].values[0]
        r2_incong = incongruente['R²'].values[0]
        
        print(f"\n  {tipo}:")
        print(f"    Congruente:    R² = {r2_cong:.4f}")
        print(f"    Incongruente:  R² = {r2_incong:.4f}")
        print(f"    Diferencia:    {r2_cong - r2_incong:.4f}")
        
        if r2_cong > r2_incong:
            print(f"    ✅ Congruente mejor explicado por ideología")
        else:
            print(f"    ⚠️  Incongruente mejor explicado")

print("\n" + "="*70)

## 10. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN FINAL: MODELOS SEM - VARIABLES DE CONGRUENCIA")
print("="*70)

print("\n📊 Modelos ejecutados:")
print(f"  - Generales: {len(resultados_generales)} modelos")
print(f"  - Ballotage: {len(resultados_ballotage)} modelos")
print(f"  - Total: {len(resultados_generales) + len(resultados_ballotage)} modelos")

print("\n📈 Estadísticas generales:")
print(f"  Generales - R² promedio: {df_metricas_gen['R²'].mean():.4f}")
print(f"  Ballotage - R² promedio: {df_metricas_bal['R²'].mean():.4f}")

# Modelos significativos
modelos_sig_gen = len(df_metricas_gen[df_metricas_gen['F_pvalue'] < 0.05])
modelos_sig_bal = len(df_metricas_bal[df_metricas_bal['F_pvalue'] < 0.05])

print(f"\n  Generales - Modelos significativos (F-test): {modelos_sig_gen}/{len(resultados_generales)}")
print(f"  Ballotage - Modelos significativos (F-test): {modelos_sig_bal}/{len(resultados_ballotage)}")

print("\n📁 Archivos generados:")
print("  - SEM_Variables_Congruencia_Generales.xlsx")
print("  - SEM_Variables_Congruencia_Ballotage.xlsx")

print("\n🎯 Hipótesis probadas:")
print("  H1: ¿Progresismo predice más cambios congruentes?")
print("  H2: ¿Conservadurismo predice más cambios congruentes?")
print("  H3: ¿Congruentes mejor predichos que incongruentes?")
print("  H4: ¿Congruentes tienen mayor R²?")

print("\n💡 Interpretación:")
print("  - Congruente: Progresistas→Izq + Conservadores→Der")
print("  - Incongruente: Progresistas→Der + Conservadores→Izq")
print("  - R² alto en Congruente = ideología predice cambios consistentes")
print("  - R² alto en Incongruente = ideología predice cambios paradójicos")

print("\n" + "="*70)
print("✅ ANÁLISIS SEM COMPLETADO")
print("="*70)